#### Search Tool

- 찾고자하는 회사의 주식시장 상장여부, 주식심볼(ticker)는 무엇인지 찾아보는 툴

  > 예 : Cloudflare를 찾으면 agent가 Cloudflare가 어떤 회사인지에 대해 검색하고,  
  >  그 회사의 주식심볼(ticker), 뉴스, 현금 흐름, 주가실적 등 모든 것에 대해 찾아보는 도구를 만들어본다.

- 아래의 명령어를 입력해 ducducgo 검색엔진을 설치 후 실행하자

  ```
  pip install duckduckgo_search --upgrade
  ```

- 회사의 주가정보 검색도구  
  www.alphavantage.co 의 api를 이용함
  api key 는 .streamlit/secrets.toml 에 저장


In [11]:
import os
import requests
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

from langchain.utilities import DuckDuckGoSearchAPIWrapper


llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"

    description = """
    Use this tool to find the stock market symbol for a company.

    It takes a quary as an argument.

    Example quary : Stock Market Symbol for Apple Company.
    """

    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()

        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company. Example : AAPL, TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyincomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the daily performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me information on CloudFlare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for CloudFlare Company'}`


107.92 +17.61 (19.50%) At close: Feb 9, 2024, 4:00 PM 107.69 -0.23 (-0.21%) After-hours: Feb 9, 2024, 7:59 PM EST 19.50 ( 1D) About NET Financial Performance According to 24 analysts, the average rating for NET stock is "Hold." The 12-month stock price forecast is $82.27, which is a decrease of -23.77% from the latest price. Price Target Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. 21 analysts have issued 1-year price targets for Cloudflare's stock. Their NET share price targets range from $43.00 to $95.00. On average, they expect the company's share price to reach $70.55 in the next twelve months. This suggests that the stock has a possible downside of 13.5%. Petr Huřťák February 8, 2024 at 4:22 PM · 5 min read Cloudflare (NYSE:NET) Q4: Be

{'input': "Give me information on CloudFlare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "Here's the information on Cloudflare Inc. (NET):\n\n1. Company Overview:\n   - CloudFlare, Inc. operates a cloud platform that offers a range of network services to companies around the world. The company is headquartered in San Francisco, California.\n   - Market Capitalization: $35,228,217,000\n   - Revenue (TTM): $1,296,745,000\n   - Gross Profit (TTM): $742,631,000\n   - EBITDA: -$49,665,000\n   - EPS (Diluted): -$0.55\n   - Price to Sales Ratio (TTM): 27.17\n   - Price to Book Ratio: 46.17\n   - 52-Week High: $116\n   - 52-Week Low: $39.9\n   - Shares Outstanding: 298,089,000\n\n2. Income Statement (Recent Fiscal Years):\n   - 2022: Operating Income: -$201,203,000, Net Income: -$193,381,000\n   - 2021: Operating Income: -$127,684,000, Net Income: -$260,309,000\n   - 2020: Operating Income: -$106

c:\Users\smin9\anaconda3\envs\pjgpt\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")
